In [1]:
import pandas as pd
import os

### Get Stage 1 OCP Created Projects

In [2]:
country = 'Cote'
file_name = 'Africa_QA/{}.xlsx'.format(country)

In [3]:
def reshape_dataframe(df):    
    """
    Only keep columns with created projects information in the dataframe
    
    Parameters
    ----------
    df : pd.DataFrame
        a dataframe with format from general tab of OCP spreadsheet 
        
    Returns
    -------
    df : pd.DataFrame
    the shrinked dataframe 
    """ 
    filter_col = [col for col in df if col.startswith('Projects Created')]
    df = df[filter_col]
    df = df.dropna(axis=1,how='all')
    df = df.melt().dropna(axis=0)
    return df

In [12]:
def get_create_ocp(path):
  
  #Get all the tab name in the excel sheet
  store = []
  all_sheet = pd.ExcelFile(os.path.join(file_name))
  sheets = all_sheet.sheet_names  

  #merge the created project id in each tab
  for i in range(len(sheets)):
    if i == 0:
      df = pd.read_excel(file_name, sheet_name=sheets[i],skiprows=[0])
      df = reshape_dataframe(df)
      df_select = df.loc[:,['value']].reset_index(drop=True)
      df_select.columns = ['Projects Created']
    elif i == 1:
      df = pd.read_excel(file_name, sheet_name=sheets[i])
      df_select = df[['Created Project ID']]
      df_select.columns = ['Projects Created']
    elif i >= 2:
      df = pd.read_excel(file_name, sheet_name=sheets[i])
      df_select = df[['Projects Created']]
    
    store.append(df_select)
    df = pd.concat(store).dropna().reset_index(drop=True)

  #break projects in one cell to one project per row 
  df['Projects Created'] = df['Projects Created'].astype(str)
  special = df['Projects Created'].str.split(',').notnull()
  df.loc[special, 'Projects Created'] = df[special]['Projects Created'].str.split(',')
  df = df.explode('Projects Created')

  #Format each cell
  df['Projects Created'] = df['Projects Created'].astype(str)
  special1 = df['Projects Created'].str.strip().notnull()
  df.loc[special1, 'Projects Created'] = df[special1]['Projects Created'].str.strip()

  #Drop duplicate
  df_clean = df.drop_duplicates('Projects Created').reset_index(drop=True)

  return df_clean

In [36]:
df_ocp = get_create_ocp(file_name)
df_ocp['source'] = 'OCP'
df_ocp['country'] = country
df_ocp.shape

(53, 3)

In [35]:
df_ocp.to_excel('stage1_projects.xlsx')

### Get Stage 1 DNA Created Projects

In [96]:
def get_created_dna(file_name, country):
    df = pd.read_excel(file_name, sheet_name='Stage 1')
    df_target = df[df['Country'] == country].reset_index(drop=True)
    return df_target['Projects Created']

In [97]:
dna_file_name = 'Africa_QA/Africa_DNA.xlsx'

In [98]:
dna = get_created_dna(dna_file_name, country)
dna

Series([], Name: Projects Created, dtype: object)

In [ ]:
for i in dna[0].split(','):
    print(i)